In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


In [3]:
warnings.simplefilter("ignore")

# table_1 -> 線形モデル・反比例モデル・対数モデル
# table_2 -> {table_1}のモデル・組み合わせモデル
# table_3 -> {table_2}のモデル・「問題サイズ/プロセス数」モデル・「プロセス数/問題サイズ」モデル
# table_4 -> {table_3}のモデル・「問題サイズの総乗 * プロセス数」モデル・「問題サイズの総乗 / プロセス数」モデル

date: str = "2022年4月15日"

latex_table: str = (
    returnAverageMAPEfromConditions(
        list_benchmarkNames=[
            "cg",
            "ep",
            # "ft",
            "is",
            # "lu",
            "mg",
        ],
        list_problemSizes=["A", "B", "C", "D", "E", "F"],
        list_processes=[2, 4, 8, 16, 32, 64, 128, 256],
        list_modelNames=[
            "functionName",
            "modelLin",
            "modelIp",
            "modelLog",
            "modelLinAndIp",
            "modelLinAndLog",
            "modelIpAndLin",
            "modelIpAndLog",
            "modelLogAndLin",
            "modelLogAndIp",
            "modelProcessDividedByProblemSize",
            "modelProblemSizeDividedByProcess",
            "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
            "modelInfiniteProductOfProblemSizeDividedByProcesses",
        ],
    )
    .style.set_precision(2)
    .to_latex(
        caption=f"平均MAPE_table4",
        label=f"{date}allModelsMAPE",
    )
)

print(latex_table)

latex_table: str = (
    returnAverageMAPEfromConditions(
        list_benchmarkNames=[
            "cg",
            "ep",
            # "ft",
            "is",
            # "lu",
            "mg",
        ],
        list_problemSizes=["A", "B", "C", "D", "E", "F"],
        list_processes=[2, 4, 8, 16, 32, 64, 128, 256],
        list_modelNames=[
            "functionName",
            "modelLin",
            "modelIp",
            "modelLog",
            "modelLinAndIp",
            "modelLinAndLog",
            "modelIpAndLin",
            "modelIpAndLog",
            "modelLogAndLin",
            "modelLogAndIp",
            "modelProcessDividedByProblemSize",
            "modelProblemSizeDividedByProcess",
            # "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
            # "modelInfiniteProductOfProblemSizeDividedByProcesses"
        ],
    )
    .style.set_precision(2)
    .to_latex(
        caption=f"平均MAPE_table3",
        label=f"{date}allModelsMAPE",
    )
)

print(latex_table)

latex_table: str = (
    returnAverageMAPEfromConditions(
        list_benchmarkNames=[
            "cg",
            "ep",
            # "ft",
            "is",
            # "lu",
            "mg",
        ],
        list_problemSizes=["A", "B", "C", "D", "E", "F"],
        list_processes=[2, 4, 8, 16, 32, 64, 128, 256],
        list_modelNames=[
            "functionName",
            "modelLin",
            "modelIp",
            "modelLog",
            "modelLinAndIp",
            "modelLinAndLog",
            "modelIpAndLin",
            "modelIpAndLog",
            "modelLogAndLin",
            "modelLogAndIp",
            # "modelProcessDividedByProblemSize",
            # "modelProblemSizeDividedByProcess",
            # "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
            # "modelInfiniteProductOfProblemSizeDividedByProcesses"
        ],
    )
    .style.set_precision(2)
    .to_latex(
        caption=f"平均MAPE_table2",
        label=f"{date}allModelsMAPE",
    )
)

print(latex_table)

latex_table: str = (
    returnAverageMAPEfromConditions(
        list_benchmarkNames=[
            "cg",
            "ep",
            # "ft",
            "is",
            # "lu",
            "mg",
        ],
        list_problemSizes=["A", "B", "C", "D", "E", "F"],
        list_processes=[2, 4, 8, 16, 32, 64, 128, 256],
        list_modelNames=[
            "functionName",
            "modelLin",
            "modelIp",
            "modelLog",
            # "modelLinAndIp",
            # "modelLinAndLog",
            # "modelIpAndLin",
            # "modelIpAndLog",
            # "modelLogAndLin",
            # "modelLogAndIp",
            # "modelProcessDividedByProblemSize",
            # "modelProblemSizeDividedByProcess",
            # "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
            # "modelInfiniteProductOfProblemSizeDividedByProcesses"
        ],
    )
    .style.set_precision(2)
    .to_latex(
        caption=f"平均MAPE_table1",
        label=f"{date}allModelsMAPE",
    )
)

print(latex_table)

warnings.resetwarnings()

\begin{table}
\caption{平均MAPE_table4}
\label{2022年4月15日allModelsMAPE}
\begin{tabular}{llr}
 & benchmarkName & averageMAPE \\
0 & cg & 17.47 \\
1 & ep & 0.00 \\
2 & is & 5.07 \\
3 & mg & 4.66 \\
\end{tabular}
\end{table}

\begin{table}
\caption{平均MAPE_table3}
\label{2022年4月15日allModelsMAPE}
\begin{tabular}{llr}
 & benchmarkName & averageMAPE \\
0 & cg & 24.75 \\
1 & ep & 0.00 \\
2 & is & 5.07 \\
3 & mg & 4.88 \\
\end{tabular}
\end{table}

\begin{table}
\caption{平均MAPE_table2}
\label{2022年4月15日allModelsMAPE}
\begin{tabular}{llr}
 & benchmarkName & averageMAPE \\
0 & cg & 46.16 \\
1 & ep & 0.00 \\
2 & is & 177.00 \\
3 & mg & 4.88 \\
\end{tabular}
\end{table}

\begin{table}
\caption{平均MAPE_table1}
\label{2022年4月15日allModelsMAPE}
\begin{tabular}{llr}
 & benchmarkName & averageMAPE \\
0 & cg & 46.19 \\
1 & ep & 0.00 \\
2 & is & 424.42 \\
3 & mg & 4.99 \\
\end{tabular}
\end{table}

